In [4]:
pip install bitstring

Note: you may need to restart the kernel to use updated packages.


In [3]:
#Import libraries
import numpy as np
import pandas as pd
import time, datetime
from datetime import datetime
from time import strftime
from time import gmtime
import bitstring
from bitstring import BitArray
import os

In [5]:
# Define repo AV
repoData = 'C:/Users/Consultant/Desktop/Raw Data/AV/'
listfile = os.listdir(repoData)

In [256]:
# Define other repos - Ex: dcu1_VE1NV11
name_rep = 'dcu3_VI1NV17/'
repoData = 'C:/Users/Consultant/Desktop/Raw Data/' + name_rep
listfile = os.listdir(repoData)

In [12]:
#Used in cycling_write_out
def find_all(s, c):
    index_list = []
    idx = s.find(c)
    index_list.append(idx)
    
    while idx != -1:        
        idx = s.find(c, idx + 1)
        index_list.append(idx)

    return index_list[:-1]

In [13]:
### Change format from 0bxxx (with  mask) to 00010100 (without mask)
def octetfull(s):

    buffer = '0'*(8-(len(s)-2))
    return buffer+s[2:]

In [14]:
# Based on table 3.3 of documentation. return the number of byte need for a specific type of data.
def switchtype(x):
    return {
        0: 1,
        1: 1,
        2: 8,
        3: 8,
        4: 16,
        5: 16,
        6: 32,
        7: 32,
        8: 64,
        9: 64,
        10: 32,
        11: 64,
    }[x]

In [15]:
# Class which initializes and structures the binary data (Input : BinFile, Output : Understable Data )
class InputData:    
    
    # Splits Data in 3 Blocks : Header, Body and Data
    def __init__(self, strbyte):
        self.header = strbyte[0:26]
        self.body = strbyte[64:2216]
        self.footer = strbyte[4160:-1]
       
    def header_decode(self):
        self.header_info = []
        #assign nth (byte -> int) to an attribute 
        self.cbm_version = int.from_bytes(self.header[0:2], byteorder='little', signed=False)
        self.header_info.append(['Version du CBM', self.cbm_version])
        self.nb_oct_allocated_IOs = int.from_bytes(self.header[2:3], byteorder='little', signed=False)
        self.header_info.append(['Nombre Octets Alloués pour les IOs', self.nb_oct_allocated_IOs])
        self.nb_oct_allocated_analogs = int.from_bytes(self.header[3:4], byteorder='little', signed=False)
        self.header_info.append(['Nombre Octets Alloués pour les Analogs', self.nb_oct_allocated_analogs])
        self.nb_vars_echantillon = int.from_bytes(self.header[4:6], byteorder='little', signed=False)
        self.header_info.append(['Nombre de Variables Echantillons', self.nb_vars_echantillon])
        self.nb_vars_EnteteSpec = int.from_bytes(self.header[6:8], byteorder='little', signed=False)
        self.header_info.append(['Nombre de Variables Entêtes Specifiques', self.nb_vars_EnteteSpec])
        self.cycle_writing_area_size = int.from_bytes(self.header[8:12], byteorder='little', signed=False)
        self.header_info.append(['Taille de la zone de data (octet)', self.cycle_writing_area_size])
        self.nb_writing = int.from_bytes(self.header[12:16], byteorder='little', signed=False)
        self.header_info.append(['Nombre Enregistrements', self.nb_writing])
        self.cbm_start_time = int.from_bytes(self.header[16:24], byteorder='little', signed=False)
        self.header_info.append(['Temps de démarrage (s)', self.cbm_start_time])
        self.cbm_acquisition_time = int.from_bytes(self.header[24:26], byteorder='little', signed=False)
        self.header_info.append(['Temps entre deux échantillons (ms)', self.cbm_acquisition_time])
    
    def body_decode(self):
        self.sample_variable = []
        self.spec_header_variable = []
        
        #\xca are delimiter for each sample variable - A changer (Ne fonctionne pas si \xca est dans entête spécifique /!\)
        self.body_list = self.body.split(b'\xca')
        
        if(len(self.body_list) > 52):
            self.body_list = self.body_list[0:52]
            self.body_list[50] = self.body_list[50] + self.body_list[51]
            self.body_list[51] = b''
            
        
        #with this operation, we have a list of byte array [sample variable 1, sample variable 2 ... , block of spec_header variable, empty byte array] )
        #The delimiter of each element of spec_header is \xfe, again, the last element of the string is a empty byte array. 
        #Therefore, we dont considere the last element (-2 instead of -1)
        
        body_list_sample = self.body_list[:-2] 
        body_list_header = self.body_list[-2].split(b'\xfe')[:-1] 
        
        
        self.datasize = []
        self.data_oct_position = []
        self.data_byte_position = []
        
        for element in body_list_sample:
            subframe = []
            subframe.append(['size',element[0]])
            subframe.append(['name',element[1:-3].decode('latin-1')])
            subframe.append(['type',element[-3]])
            subframe.append(['bitposition',element[-2]])
            subframe.append(['sampleposition',element[-1]])
            
            if element[-1] < self.nb_oct_allocated_IOs:
                subframe.append(['signal_type','IO'])
            elif element[-1] < (self.nb_oct_allocated_analogs + self.nb_oct_allocated_IOs):
                subframe.append(['signal_type','Analog'])
            else:
                subframe.append(['signal_type','File error'])
            
            self.datasize.append(switchtype(element[-3]))
            self.data_oct_position.append(element[-1])
            self.data_byte_position.append(element[-2])
            
            self.sample_variable.append(subframe)
            
        for element in body_list_header:
            subframe = []
            subframe.append(['size',element[0]])
            
            #the name of each variable end with \x00
            index = element.find(b'\x00')
            subframe.append(['name',element[1:index+1].decode('latin-1')])
            subframe.append(['type',element[index+1]])
            subframe.append(['sizevalue',element[index+2]])
            subframe.append(['value',element[index+3:index+3 + element[index+2]]])
            
            self.spec_header_variable.append(subframe)

            
    def footer_decode(self):
        self.measurement_data = []
        footer_size = len(self.footer)
        
        
        #each frame has at least 5 character
        min_size_per_frame = 5
        
        index = 0
        while index < self.cycle_writing_area_size :
            
            frame = []
            subframe = []
                                    
            frame.append(['U8Transition',hex(self.footer[index])[-2]])
            frame.append(['U8SleepMode',hex(self.footer[index])[-1]])
            self.ms = int.from_bytes(self.footer[index+1:index+4],byteorder='little', signed=False)
            s = self.ms / 1000
            timestamp = strftime(("%H:%M:%S"), gmtime(s))
            frame.append(['U24TimeStamp', s])
            
            ### int -> bin -> str ---> filter passing the 7 last byte ---> count number of 1
            control_byte = bin(self.footer[index+4]) #defines control byte
            nb_control_byte = str(control_byte)[1:].count("1") #counts number of 1 in control byte to define auZipDBitData
            frame.append(['u8ZipControlBits',nb_control_byte, control_byte])
            
            #print('Control Byte = ' + control_byte)
            
            buffer = '0'*8
            
            data_bit = ''
        
            flip_control_byte = control_byte[::-1]
            pointer = 0
            for i in range(len(flip_control_byte)-2):
                
                
                if flip_control_byte[i] == '0':
                
                    data_bit =  buffer + data_bit
                    
                else:
                    
                    ### I cant convert data directly to hex to binary, hence I do
                    ### hex -> integer -> binary  ---> count number of 1
                    data_oct = bin(int.from_bytes(self.footer[index+5+pointer:index+5+pointer+1], byteorder='little', signed=False))
                    data_oct = octetfull(data_oct)                
                    
                    pointer = pointer +1
                    data_bit = data_oct + data_bit
                                         
            #print('ZipBitData = ' + data_bit)               
                  
            nb_data_bit = data_bit.count('1') #counts number of 1 in auZipBitData         
            frame.append(['auZipBitData',nb_data_bit,data_bit])
            
            subindex = 0
            while subindex < nb_data_bit :
                subframe_bin = octetfull(bin(self.footer[index+min_size_per_frame +nb_control_byte+subindex]))
                subframe.append([subframe_bin])
                subindex = subindex + 1
                
            frame.append(["measurement",subframe])
            self.measurement_data.append(frame)
            index = index + min_size_per_frame  + nb_control_byte + subindex
    
    def get_metadata(self, name):
        
        self.metadata_finaldata = pd.DataFrame(columns={})
        
        ID = []
        ID.append(name.replace('.bin',''))
        self.metadata_finaldata['ID'] = ID
        
        date_dep = []
        date_dep.append(datetime.fromtimestamp(self.header_info[7][1]).strftime("%A, %B %d, %Y %I:%M:%S %p"))
        self.metadata_finaldata['Date Depart'] = date_dep
        
        label1 = self.spec_header_variable[0][1][1].replace('\x00','')
        value1 = []
        value1.append(str(self.spec_header_variable[0][4][1].decode('ascii')))
        self.metadata_finaldata[label1] = str(value1)
        
        label2 = self.spec_header_variable[1][1][1].replace('\x00','')
        value2 = []
        value2.append(int.from_bytes(self.spec_header_variable[1][4][1], byteorder='little', signed=False))
        self.metadata_finaldata[label2] = value2
        
        label3 = self.spec_header_variable[2][1][1].replace('\x00','')
        value3 = []
        value3.append(int.from_bytes(self.spec_header_variable[2][4][1], byteorder='little', signed=False))
        self.metadata_finaldata[label3] = value3
        
        label4 = self.spec_header_variable[3][1][1].replace('\x00','')
        self.value4 = []
        self.value4.append(str(self.spec_header_variable[3][4][1].decode('utf-8')))
        self.metadata_finaldata[label4] = str(self.value4)
        
        label5 = self.spec_header_variable[4][1][1].replace('\x00','')
        value5 = []
        value5.append(int.from_bytes(self.spec_header_variable[4][4][1], byteorder='little', signed=False))
        self.metadata_finaldata[label5] = value5
        
        label6 = self.spec_header_variable[5][1][1].replace('\x00','')
        value6 = []
        value6.append(int.from_bytes(self.spec_header_variable[5][4][1], byteorder='little', signed=False))
        self.metadata_finaldata[label6] = value6
        
        label7 = self.spec_header_variable[6][1][1].replace('\x00','') + '(en degré)'
        value7 = []
        value7.append(int.from_bytes(self.spec_header_variable[6][4][1], byteorder='little', signed=False)/10)
        self.metadata_finaldata[label7] = value7
        
        for i in range(len(self.header_info)):
            info = self.header_info[i][1]
            self.metadata_finaldata[self.header_info[i][0]] = info
        
        
    def cycling_write_out(self):
        
        # we build an empty matrix for ios and analog data. 
        dataline = "0"*self.nb_oct_allocated_IOs*8 + "0"*self.nb_oct_allocated_analogs*8
        self.sample = pd.DataFrame(columns={'ID','Sample','Sub Sample'})
        self.sample_list = []
        self.id_list = []
        
        for i in range(self.nb_writing):
        
                #find all the position of 1 in the octets 
                #octet is read from right to left.
                index_list = find_all(self.measurement_data[i][4][-1][::-1], '1')
   
                for j in range(len(index_list)):
                    
                    ## data:  int -> bin -> fullbin -> flip because the bin is read from left to right
                    new_write = self.measurement_data[i][5][-1][j][0]
                    
                    ### rewrite the chosen octet
                    dataline = dataline[0:index_list[j]*8] + new_write[::-1] + dataline[index_list[j]*8+8:]
                    #print(self.measurement_data[i][4][1])
                    #print('Data = ' + dataline)
                
                self.id_list.append(i)
                self.sample_list.append(dataline)
                
        self.sample['ID'] = self.id_list
        self.sample['Sample'] = self.sample_list
        
        self.final_data = pd.DataFrame(columns={'TimeStamp'})
        timestamp = []
        sleepmode = []
        transition = []
        ind = 0
        while ind < self.nb_writing :
            timestamp.append(self.measurement_data[:][ind][2][1])
            sleepmode.append(self.measurement_data[:][ind][1][1])
            transition.append(self.measurement_data[:][ind][0][1])
            ind = ind + 1
        self.final_data['TimeStamp'] = timestamp
        self.final_data['Sleepmode'] = sleepmode
        self.final_data['Transition'] = transition
        
        var1 = []
        var2 = []
        var3 = []
        var4 = []
        var5 = []
        var6 = []
        var7 = []
        var8 = []
        var9 = []
        var10 = []
        var11 = []
        var12 = []
        var13 = []
        var14 = []
        var15 = []
        var16 = []
        var17 = []
        var18 = []
        var19 = []
        var20 = []
        var21 = []
        var22 = []
        var23 = []
        var24 = []
        var25 = []
        var26 = []
        var27 = []
        var28 = []
        var29 = []
        var30 = []
        var31 = []
        var32 = []
        var33 = []
        var34 = []
        var35 = []
        var36 = []
        var37 = []
        var38 = []
        var39 = []
        var40 = []
        var41 = []
        var42 = []
        var43 = []
        var44 = []
        var45 = []
        var46 = []
        var47 = []
        var48 = []
        var49 = []
        var50 = []
        var51 = []
        
        
        for k in range(len(self.sample_list)):
            
            var1.append(self.sample_list[k][0]) 
            var2.append(self.sample_list[k][1]) 
            var3.append(self.sample_list[k][2])
            var4.append(self.sample_list[k][3])
            var5.append(self.sample_list[k][4])
            var6.append(self.sample_list[k][5])
            var7.append(self.sample_list[k][6])
            var8.append(self.sample_list[k][7])
            var9.append(self.sample_list[k][8])
            var10.append(self.sample_list[k][9]) 
            var11.append(self.sample_list[k][10]) 
            var12.append(self.sample_list[k][11]) 
            var13.append(self.sample_list[k][12]) 
            var14.append(self.sample_list[k][13]) 
            var15.append(self.sample_list[k][14]) 
            var16.append(self.sample_list[k][15]) 
            var17.append(self.sample_list[k][16]) 
            var18.append(self.sample_list[k][17]) 
            var19.append(self.sample_list[k][18]) 
            var20.append(self.sample_list[k][19]) 
            var21.append(self.sample_list[k][20]) 
            var22.append(self.sample_list[k][21]) 
            var23.append(self.sample_list[k][22]) 
            var24.append(self.sample_list[k][23]) 
            var25.append(self.sample_list[k][24]) 
            var26.append(self.sample_list[k][25]) 
            var27.append(self.sample_list[k][26]) 
            var28.append(self.sample_list[k][27]) 
            var29.append(self.sample_list[k][28]) 
            var30.append(self.sample_list[k][29]) 
            var31.append(self.sample_list[k][30]) 
            var32.append(self.sample_list[k][31]) 
            var33.append(self.sample_list[k][32]) 
            var34.append(self.sample_list[k][33]) 
            var35.append(self.sample_list[k][34]) 
            var36.append(self.sample_list[k][35]) 
            var37.append(self.sample_list[k][36]) 
            var38.append(self.sample_list[k][37]) 
            var39.append(self.sample_list[k][38]) 
            var40.append(self.sample_list[k][39]) 
            var41.append(self.sample_list[k][40]) 
            var42.append(self.sample_list[k][41]) 
            var43.append(self.sample_list[k][42]) 
            var44.append(int(self.sample_list[k][48:112],2))
            cp= self.sample_list[k][112:128][::-1]
            courant_mot_porte = BitArray(bin=cp).int
            var45.append(courant_mot_porte) 
            tp= self.sample_list[k][128:144][::-1]
            tension_mot_porte = BitArray(bin=tp).int
            var46.append(tension_mot_porte)
            pp= self.sample_list[k][144:160][::-1]
            position_mot_porte = BitArray(bin=pp).int
            var47.append(position_mot_porte) 
            cm= self.sample_list[k][160:176][::-1]
            courant_mot_marche = BitArray(bin=cm).int
            var48.append(courant_mot_marche)
            tm= self.sample_list[k][176:192][::-1]
            tension_mot_marche = BitArray(bin=tm).int
            var49.append(tension_mot_marche)
            pm= self.sample_list[k][192:208][::-1]
            position_mot_marche = BitArray(bin=pm).int
            var50.append(position_mot_marche)
            
        index = 0
        while index < len(self.sample_variable):
            label = self.sample_variable[index][1][1].replace('\x00','')
            self.final_data[label] = 0
            index = index + 1
        
        #Set data in Final DataFrame
        self.final_data[self.sample_variable[0][1][1].replace('\x00','')] = var1
        self.final_data[self.sample_variable[1][1][1].replace('\x00','')] = var2
        self.final_data[self.sample_variable[2][1][1].replace('\x00','')] = var3  
        self.final_data[self.sample_variable[3][1][1].replace('\x00','')] = var4
        self.final_data[self.sample_variable[4][1][1].replace('\x00','')] = var5
        self.final_data[self.sample_variable[5][1][1].replace('\x00','')] = var6
        self.final_data[self.sample_variable[6][1][1].replace('\x00','')] = var7
        self.final_data[self.sample_variable[7][1][1].replace('\x00','')] = var8  
        self.final_data[self.sample_variable[8][1][1].replace('\x00','')] = var9
        self.final_data[self.sample_variable[9][1][1].replace('\x00','')] = var10
        self.final_data[self.sample_variable[10][1][1].replace('\x00','')] = var11
        self.final_data[self.sample_variable[11][1][1].replace('\x00','')] = var12
        self.final_data[self.sample_variable[12][1][1].replace('\x00','')] = var13 
        self.final_data[self.sample_variable[13][1][1].replace('\x00','')] = var14
        self.final_data[self.sample_variable[14][1][1].replace('\x00','')] = var15
        self.final_data[self.sample_variable[15][1][1].replace('\x00','')] = var16
        self.final_data[self.sample_variable[16][1][1].replace('\x00','')] = var17
        self.final_data[self.sample_variable[17][1][1].replace('\x00','')] = var18  
        self.final_data[self.sample_variable[18][1][1].replace('\x00','')] = var19
        self.final_data[self.sample_variable[19][1][1].replace('\x00','')] = var20
        self.final_data[self.sample_variable[20][1][1].replace('\x00','')] = var21
        self.final_data[self.sample_variable[21][1][1].replace('\x00','')] = var22
        self.final_data[self.sample_variable[22][1][1].replace('\x00','')] = var23
        self.final_data[self.sample_variable[23][1][1].replace('\x00','')] = var24 
        self.final_data[self.sample_variable[24][1][1].replace('\x00','')] = var25
        self.final_data[self.sample_variable[25][1][1].replace('\x00','')] = var26
        self.final_data[self.sample_variable[26][1][1].replace('\x00','')] = var27
        self.final_data[self.sample_variable[27][1][1].replace('\x00','')] = var28
        self.final_data[self.sample_variable[28][1][1].replace('\x00','')] = var29  
        self.final_data[self.sample_variable[29][1][1].replace('\x00','')] = var30
        self.final_data[self.sample_variable[30][1][1].replace('\x00','')] = var31
        self.final_data[self.sample_variable[31][1][1].replace('\x00','')] = var32
        self.final_data[self.sample_variable[32][1][1].replace('\x00','')] = var33
        self.final_data[self.sample_variable[33][1][1].replace('\x00','')] = var34  
        self.final_data[self.sample_variable[34][1][1].replace('\x00','')] = var35
        self.final_data[self.sample_variable[35][1][1].replace('\x00','')] = var36
        self.final_data[self.sample_variable[36][1][1].replace('\x00','')] = var37
        self.final_data[self.sample_variable[37][1][1].replace('\x00','')] = var38
        self.final_data[self.sample_variable[38][1][1].replace('\x00','')] = var39  
        self.final_data[self.sample_variable[39][1][1].replace('\x00','')] = var40
        self.final_data[self.sample_variable[40][1][1].replace('\x00','')] = var41
        self.final_data[self.sample_variable[41][1][1].replace('\x00','')] = var42
        self.final_data[self.sample_variable[42][1][1].replace('\x00','')] = var43
        self.final_data[self.sample_variable[43][1][1].replace('\x00','')] = var44  
        self.final_data[self.sample_variable[44][1][1].replace('\x00','')] = var45
        self.final_data[self.sample_variable[45][1][1].replace('\x00','')] = var46
        self.final_data[self.sample_variable[46][1][1].replace('\x00','')] = var47
        self.final_data[self.sample_variable[47][1][1].replace('\x00','')] = var48
        self.final_data[self.sample_variable[48][1][1].replace('\x00','')] = var49  
        self.final_data[self.sample_variable[49][1][1].replace('\x00','')] = var50
        

In [62]:
#Init log list and error list
log = []
err = []

#Export Files Decoded as CSV - NAT
for i in range(len(listfile)):
    filename = listfile[i]  
    
    #Load data
    f = open(repoData + filename,"rb")
    
    # Read Data and Store Binary Data
    strbyte = f.read()
    f.close()
    
    if(bool(strbyte) == False):
        log.append('Le fichier ' + listfile[i] + ' est vide' + ' | ' + 'Time = ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S %p'))
        err.append('Le fichier ' + listfile[i] + ' est vide' + ' | ' + 'Time = ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S %p'))
    else:
        data = InputData(strbyte)

        data.header_decode()
        data.body_decode()
        data.footer_decode() 
        data.get_metadata(filename)
        data.cycling_write_out()

        listname = []
        listname.append(filename.replace('.bin',''))
        data.final_data['ID'] = filename.replace('.bin','')

        #Give name to metadata csv file
        namecsv1 = 'C:/Users/Consultant/Desktop/Decoded Data/AV/metadata/' + filename.replace('.bin','_metadata.csv')
        data.metadata_finaldata.to_csv(namecsv1, index=False)

        #Give name to data csv file
        namecsv2 = 'C:/Users/Consultant/Desktop/Decoded Data/AV/data/' + filename.replace('.bin','.csv')
        data.final_data.to_csv(namecsv2, index=False)
        
        log.append('Le fichier ' + listfile[i] + ' a été créer avec succès' + ' | ' + 'Time = ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S %p'))

#Create and write log file
k= open('C:/Users/Consultant/Desktop/Decoded Data/AV/log/log.txt',"w+")
for i in range(len(log)):
    k.write(log[i] + "\n")
k.close()

#Create and write error file
p= open('C:/Users/Consultant/Desktop/Decoded Data/AV/log/error.txt',"w+")
for i in range(len(err)):
    p.write(err[i] + "\n")
p.close()  

In [258]:
#Init log list and error list
log = []
err = []

#Export Files Decoded as CSV - dcu1_VE1NV11
for i in range(len(listfile)):
    filename = listfile[i]  
    
    #Load data
    f = open(repoData + filename,"rb")
    
    # Read Data and Store Binary Data
    strbyte = f.read()
    f.close()
    
    if(bool(strbyte) == False):
        log.append('Le fichier ' + listfile[i] + ' est vide' + ' | ' + 'Time = ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S %p'))
        err.append('Le fichier ' + listfile[i] + ' est vide' + ' | ' + 'Time = ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S %p'))
    else:
        data = InputData(strbyte)
        
        data.header_decode()
        if(data.body[-2:-1] != b'\xfe'):
            log.append('Le fichier ' + listfile[i] + ' semble corrompu' + ' | ' + 'Time = ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S %p'))
            err.append('Le fichier ' + listfile[i] + ' semble corrompu' + ' | ' + 'Time = ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S %p'))
        elif(len(data.footer) == 0):
            log.append('Le fichier ' + listfile[i] + ' semble corrompu' + ' | ' + 'Time = ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S %p'))
            err.append('Le fichier ' + listfile[i] + ' semble corrompu' + ' | ' + 'Time = ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S %p'))
        else:
            print(listfile[i])
            data.body_decode()
            data.footer_decode() 
            data.get_metadata(filename)
            data.cycling_write_out()

            listname = []
            listname.append(filename.replace('.bin',''))
            data.final_data['ID'] = filename.replace('.bin','')

            #Give name to metadata csv file
            namecsv1 = 'C:/Users/Consultant/Desktop/Decoded Data/' + name_rep + 'metadata/' + filename.replace('.bin','_metadata.csv')
            data.metadata_finaldata.to_csv(namecsv1, index=False)

            #Give name to data csv file
            namecsv2 = 'C:/Users/Consultant/Desktop/Decoded Data/' + name_rep + 'data/' + filename.replace('.bin','.csv')
            data.final_data.to_csv(namecsv2, index=False)
            
            log.append('Le fichier ' + listfile[i] + ' a été créer avec succès' + ' | ' + 'Time = ' + datetime.now().strftime('%Y-%m-%d %H:%M:%S %p'))


#Create or overwrites log file
k= open('C:/Users/Consultant/Desktop/Decoded Data/' + name_rep + 'log/log.txt',"w+")
for i in range(len(log)):
    k.write(log[i] + "\n")
k.close()

#Create or overwrites error file
p= open('C:/Users/Consultant/Desktop/Decoded Data/' + name_rep + 'log/error.txt',"w+")
for i in range(len(err)):
    p.write(err[i] + "\n")
p.close()  

z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_032542.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_034815.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_035145.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_040049.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_041509.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_043655.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_045935.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_050437.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_051011.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_051424.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_051939.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190409_052518.bin
z5700003________

z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_042805.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_043354.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_181708.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_182343.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_183747.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_185500.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_190230.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_192113.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_195744.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_195925.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_200120.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190415_200159.bin
z5700003________

z5700003________zr_57_83003_____dcu3____________dcucbmdata______190505_073623.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190505_074845.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190505_221828.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190505_222833.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190505_223042.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190505_225842.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190505_232335.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190505_232923.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190505_233620.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190505_234156.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190510_054820.bin
z5700003________zr_57_83003_____dcu3____________dcucbmdata______190510_065344.bin
z5700003________

z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_063620.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_064926.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_140407.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_141317.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_142118.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_142517.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_143218.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_143855.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_144518.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_154638.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_155810.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190503_162059.bin
z5700005________

z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_065135.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_065647.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_070039.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_070354.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_074750.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_075158.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_075504.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_075846.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_080325.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_080555.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_081036.bin
z5700005________zr_57_83005_____dcu3____________dcucbmdata______190519_081508.bin
z5700005________

...

z5700033________zr_57_83033_____dcu3____________dcucbmdata______190513_085446.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190513_085831.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190513_090436.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190513_090905.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190513_091439.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190513_091920.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190514_163342.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190517_022430.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190517_023121.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190517_023325.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190521_144512.bin
z5700033________zr_57_83033_____dcu3____________dcucbmdata______190521_144635.bin
z5700033________

z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_062107.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_064409.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_065016.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_065716.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_070155.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_070630.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_071306.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_071701.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_072425.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_072919.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_073557.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190410_074131.bin
z5700053________

z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_095637.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_100036.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_100459.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_101104.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_101707.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_111715.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_114326.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_114939.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_115630.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_120107.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_120540.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190411_121229.bin
z5700053________

z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_044938.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_045510.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_050100.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_050511.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_050926.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_051514.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_051845.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_052410.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_052855.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_053504.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_054030.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190417_054626.bin
z5700053________

z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_193708.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_194324.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_194734.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_195508.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_195914.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_200411.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_201101.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_201711.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_204245.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_211930.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_214123.bin
z5700053________zr_57_83053_____dcu3____________dcucbmdata______190418_221517.bin
z5700053________